<center>

### COSC2753 - Machine Learning

# **Logistic Regression**

<center>────────────────────────────</center>
&nbsp;


# I. Global Configuration

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import pandas as pd

# II. Data Loading

In [9]:
# df = pd.read_csv("../data/processed/data_train_processed.csv")
df = pd.read_csv("../../data/processed/data_train_processed.csv")

# Split features and target variable
X = df.drop(columns=["Status"])
y = df["Status"]

# III. Model Development

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
poly_degree = 2

# Create pipeline
pipeline = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=poly_degree)),
        ("logistic", LogisticRegression(max_iter=1000)),
    ]
)

# Training the model
pipeline.fit(X_train, y_train)

# Evaluation
train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

# Accuracy
print("Training Accuracy Score:", train_score)
print("Testing Accuracy Score:", test_score)

# Show classification report
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

c:\Users\huuqu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training Accuracy Score: 0.7846675742796445
Testing Accuracy Score: 0.7857035995332277
              precision    recall  f1-score   support

           0       0.77      0.81      0.79     33504
           1       0.80      0.76      0.78     33338

    accuracy                           0.79     66842
   macro avg       0.79      0.79      0.79     66842
weighted avg       0.79      0.79      0.79     66842

